In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'trajectory_FC_large'

In [3]:
config_path = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME}.yaml")
config = load_config(config_path)

In [4]:
model = CondRealNVP.from_config(config).to(device)

print(f'{model.n_params:,}')

48,865,045


In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/computerman/Desktop/IGNNS/IGNNS-final-project/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 1/1 [00:00<00:00, 1178.51it/s, file=test]

Using trajectories data for training. Shapes:
X shape: torch.Size([10, 30, 3])
y shape: torch.Size([10, 21])


In [6]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ichich111222333 (balisticcnf). Use `wandb login --relogin` to force relogin


  0%|          | 0/50000 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/train/trainer.py", line 100, in train
    model = self._train(
            ^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/train/trainer.py", line 227, in _train
    loss = self._train_batch(x, y, model, optimizer, loss_function)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/train/trainer.py", line 306, in _train_batch
    z = model.forward(y, x, log_det_J=True)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/src/bcnf/models/cnf.py", line 306, in forward
    x = layer(x, log_det_J)
        ^^^^^^^^^^^^^^^^^^^
  File "/home/computerman/Desktop/IGNNS/IGNNS-final-project/venv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
  

RuntimeError: The size of tensor a (19) must match the size of tensor b (21) at non-singleton dimension 1

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/trajectory_FC_large
